# **Sistema de Recomendación de Peliculas**
El EDA debería incluir gráficas interesantes para extraer datos, como por ejemplo una nube de palabras 
con las palabras más frecuentes en los títulos de las películas. 
Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se 
debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según 
el score de similaridad y devolverá una lista de Python con 5 valores, cada uno siendo el string del 
nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función 
adicional de la API anterior y debe llamarse:

+ def **recomendacion( *`titulo`*)**: Se ingresa el nombre de una película y te recomienda las similares en una 
lista de 5 valores.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#import networkx as nx
import time
import seaborn as sns
import re
import math
sns.set_style("whitegrid")
#sns.set(style="darkgrid")
sns.set_palette("tab10")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import datetime as dt
plt.style.use('seaborn-notebook')
params = {'legend.fontsize': 16,
          'legend.title_fontsize': 16,
          'figure.figsize': (15, 5),
          'axes.labelsize': 18,
          'axes.titlesize': 20,
          'xtick.labelsize': 18,
          'ytick.labelsize': 18}
plt.rcParams.update(params)
ing_fmt = 'svg'

import warnings
warnings.filterwarnings("ignore")

C:\Users\Asus\AppData\Local\Temp\ipykernel_14940\1256747621.py:19: MatplotlibDeprecationWarning:

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.



In [16]:
movie_df = pd.read_csv('movies_dataset.csv')
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45378 entries, 0 to 45377
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  45378 non-null  object 
 1   budget                 45378 non-null  float64
 2   genres                 45378 non-null  object 
 3   id                     45378 non-null  int64  
 4   original_language      45378 non-null  object 
 5   overview               45378 non-null  object 
 6   popularity             45378 non-null  float64
 7   production_companies   45378 non-null  object 
 8   production_countries   45378 non-null  object 
 9   release_date           45378 non-null  object 
 10  revenue                45378 non-null  float64
 11  runtime                45378 non-null  float64
 12  spoken_languages       45378 non-null  object 
 13  status                 45378 non-null  object 
 14  tagline                45378 non-null  object 
 15  ti

In [24]:
movie_df = pd.read_csv('movies_dataset.csv')
df = movie_df[['belongs_to_collection','genres','id','overview','title','vote_average']].sort_values('id')
df.reset_index(drop=False,inplace=True)
df = df[0:30000]
df.head()

,index,belongs_to_collection,genres,id,overview,title,vote_average
0,43858,'Sin Dato','Drama',2,Taisto Kasurinen is a Finnish coal miner whose...,Ariel,7.1
1,43900,'Sin Dato','Drama',3,"An episode in the life of Nikander, a garbage ...",Shadows in Paradise,7.1
2,17,'Sin Dato','Crime',5,It's Ted the Bellhop's first night on the job....,Four Rooms,6.5
3,436,'Sin Dato','Action',6,"While racing to a boxing match, Frank, Mike, J...",Judgment Night,6.4
4,232,'Star Wars Collection','Adventure',11,Princess Leia is captured and held hostage by ...,Star Wars,8.1


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Defina un objeto vectorizador TF-IDF. Elimné todas las palabras vacías en ingles,como 'the', 'a'"Material de apoyo.md"
tfidf = TfidfVectorizer(stop_words='english')

In [26]:
# Reemplazar NaN con una cadena vacía
df['overview'] = df['overview'].fillna('')

# Contruya lamatriz TF_IDF requerida ajustando y transformando los datos
tfidf_matrix = tfidf.fit_transform(df['overview'])

# Salida de la forma de tfidf_matrix
tfidf_matrix.shape

(30000, 62410)

In [27]:
# Import linear_kernel
from sklearn.metrics.pairwise import laplacian_kernel

# COmpute the cosine similarity matrix
cosine_sim = laplacian_kernel(tfidf_matrix, tfidf_matrix)

In [28]:
# Construya un mapa inverso de índices y títulos de películas.
indices =pd.Series(df.index, index=df['title']).drop_duplicates()

In [30]:
def get_recomendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    # Obtenga las puntuaciones de similitud por partes de todas las películas con esapelícula
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordene las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse=True)

    # Obtén las puntuaciones de las 10 películas más similares
    sim_scores = sim_scores[1:11]

    # Obtenga los índices de películas
    movie_indices = [i[0] for i in sim_scores]

    # Devuelve el top 10 de películas más similares
    return df['title'].iloc[movie_indices]

In [31]:
get_recomendations('Ariel')

17979                                     Iceland
19196                                         Sur
19327                          Afstiros katallilo
19498                                   Snow days
20165                          Slaves of New York
29555                                   Game Over
932                                    Metal Skin
953                       Keine Lieder über Liebe
1049                         Praxis Dr. Hasenbein
1051     Texas - Doc Snyder hält die Welt in Atem
Name: title, dtype: object